## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,2022-05-27 23:23:59,64.3833,-173.3000,43.52,72,62,8.43,broken clouds
1,1,Busselton,AU,2022-05-27 23:22:50,-33.6500,115.3333,53.87,75,46,6.15,scattered clouds
2,2,Carnarvon,AU,2022-05-27 23:32:13,-24.8667,113.6333,70.77,70,100,7.00,overcast clouds
3,3,Butaritari,KI,2022-05-27 23:23:56,3.0707,172.7902,81.05,75,66,7.52,light rain
4,4,Dehui,CN,2022-05-27 23:32:13,44.5333,125.7000,68.45,49,94,8.41,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 89


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Butaritari,KI,2022-05-27 23:23:56,3.0707,172.7902,81.05,75,66,7.52,light rain
5,5,Kapaa,US,2022-05-27 23:23:55,22.0752,-159.3190,81.12,65,100,10.36,overcast clouds
10,10,Puerto Ayora,EC,2022-05-27 23:25:08,-0.7393,-90.3518,73.54,87,52,4.00,broken clouds
13,13,Machilipatnam,IN,2022-05-27 23:32:15,16.1667,81.1333,84.20,79,6,10.07,clear sky
16,16,Lorengau,PG,2022-05-27 23:32:15,-2.0226,147.2712,83.21,62,99,4.41,overcast clouds
18,18,Aksu,CN,2022-05-27 23:32:16,41.1231,80.2644,72.21,24,0,1.59,clear sky
20,20,Beberibe,BR,2022-05-27 23:32:16,-4.1797,-38.1306,76.66,86,74,9.66,broken clouds
23,23,Shirgaon,IN,2022-05-27 23:32:17,19.7000,72.7000,83.61,67,25,14.36,scattered clouds
24,24,Biltine,TD,2022-05-27 23:32:18,14.5333,20.9167,82.76,8,1,8.12,clear sky
31,31,Atuona,PF,2022-05-27 23:29:05,-9.8000,-139.0333,78.51,75,34,19.82,scattered clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                259
City                   259
Country                259
Date                   259
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# making new data frame with dropped NA values
preferred_cities_df = preferred_cities_df.dropna(axis = 0, how ='any')

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.05,light rain,3.0707,172.7902,
5,Kapaa,US,81.12,overcast clouds,22.0752,-159.3190,
10,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,
13,Machilipatnam,IN,84.20,clear sky,16.1667,81.1333,
16,Lorengau,PG,83.21,overcast clouds,-2.0226,147.2712,
18,Aksu,CN,72.21,clear sky,41.1231,80.2644,
20,Beberibe,BR,76.66,broken clouds,-4.1797,-38.1306,
23,Shirgaon,IN,83.61,scattered clouds,19.7000,72.7000,
24,Biltine,TD,82.76,clear sky,14.5333,20.9167,
31,Atuona,PF,78.51,scattered clouds,-9.8000,-139.0333,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
#and
# 6c. Get latitude and longitude from DataFrame

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
   
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

# 6d. Set up the base URL for the Google Directions API to get JSON data.

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
 # 6e. Make request and retrieve the JSON data from the search. 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

# Get the text of the 'Get' request.


   
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
hotel_df.head()        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.05,light rain,3.0707,172.7902,Isles Sunset Lodge
5,Kapaa,US,81.12,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Machilipatnam,IN,84.20,clear sky,16.1667,81.1333,R&B Bungalow
16,Lorengau,PG,83.21,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [65]:
# 7. Drop the rows where there is no Hotel Name.
len(hotel_df)        

259

In [66]:
# Determine if there are any empty rows.

hotel_df.count()


City                   259
Country                259
Max Temp               259
Current Description    259
Lat                    259
Lng                    259
Hotel Name             259
dtype: int64

In [71]:
NULL_hotel_df= hotel_df
NULL_hotel_df= NULL_hotel_df.loc[(NULL_hotel_df["Hotel Name"] == "")]
NULL_hotel_df
NULL_hotel_df.count()

City                   16
Country                16
Max Temp               16
Current Description    16
Lat                    16
Lng                    16
Hotel Name             16
dtype: int64

In [72]:
clean_hotel_df= hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df.count()

City                   243
Country                243
Max Temp               243
Current Description    243
Lat                    243
Lng                    243
Hotel Name             243
dtype: int64

In [83]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.05,light rain,3.0707,172.7902,Isles Sunset Lodge
5,Kapaa,US,81.12,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Machilipatnam,IN,84.20,clear sky,16.1667,81.1333,R&B Bungalow
16,Lorengau,PG,83.21,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
...,...,...,...,...,...,...,...
675,Basco,PH,83.39,broken clouds,20.4487,121.9702,Nathaniel's Lodge
676,Indian Head,US,73.89,light rain,38.6001,-77.1622,"Red Carpet Inn Indian Head, MD"
677,Kuching,MY,75.42,few clouds,1.5500,110.3333,Hilton Kuching
678,Islamkot,PK,79.43,few clouds,24.6997,70.1770,Haji Hotel


In [74]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [81]:
# 9. Using the template add city name, the country code, 
# the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [84]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))